In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
# Train test split
from sklearn.model_selection import train_test_split
# Text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://cogcomp.seas.upenn.edu/Data/QA/QC/train_1000.label"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

In [3]:
from datasets import load_dataset

dataset = load_dataset("launch/open_question_type")

In [4]:
fquestions=[]
flabel=[]

In [5]:
import pandas as pd

# Assuming your DatasetDict is stored in a variable called dataset_dict

# Merge train and test datasets
merged_df = pd.concat([dataset['train'].to_pandas(), dataset['test'].to_pandas()], ignore_index=True)

# Drop unnecessary columns
merged_df.drop(columns=['id', 'annotator1', 'annotator2', 'resolve_type'], inplace=True)

In [6]:
open=0
close=0
qlist=soup.text.split("\n")

for text in qlist:
  j=text.split(" ",1)


  if(len(j)>1):
    fquestions.append(j[1])


    if(j[0].startswith("DESC")):
      flabel.append(0)
      open+=1
    else:
      flabel.append(1)
      close+=1
print(open,close)

211 789


In [7]:
fquestions.extend(list(merged_df['question'])[:578])

In [8]:
for i in range(578):
    flabel.append(0)

In [9]:
import nltk
import re
import unicodedata
import inflect
import contractions

from nltk.corpus import stopwords
#lemma = WordNetLemmatizer()






    
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english') and word.lower() not in ["why","how","when","who","what","can","could","would","should"]:
            new_words.append(word)
    words=new_words
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    words=new_words
    

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    words=new_words
    

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    words=new_words
    

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    words=new_words
    
def normalize(words):
    replace_numbers(words)
    remove_punctuation(words)
    to_lowercase(words)
    remove_non_ascii(words)
    remove_stopwords(words)
    return words
    

In [10]:
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
for i in fquestions:
    i=normalize(i)

In [12]:
print(len(fquestions),len(flabel))

1578 1578


In [13]:
import gzip
import shutil
from gensim.models import KeyedVectors

# Step 1: Decompress the .gz file
#input_file = r'C:\Users\asus\Desktop\iknowthis\finalenv\CNN TEXT\GoogleNews-vectors-negative300-SLIM.bin.gz'
#output_file = r'C:\Users\asus\Desktop\iknowthis\finalenv\CNN TEXT\word2vec_model\GoogleNews-vectors-negative300-SLIM.bin'

# Decompress the file
#with gzip.open(input_file, 'rb') as f_in:
#    with open(output_file, 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

print("File decompressed successfully")


# Step 2: Load the Word2Vec model
model_path = 'word2vec_model/GoogleNews-vectors-negative300-SLIM.bin'
embed_lookup = KeyedVectors.load_word2vec_format(model_path, binary=True)

File decompressed successfully


In [48]:
from nltk.tokenize import word_tokenize
qse = pd.Series(fquestions)
lse =pd.Series(flabel)
max_len=300
xtrain, xtest, ytrain, ytest = train_test_split(qse, lse, shuffle=True, test_size=0.2,stratify=lse)

max_words = 10000
print(type(xtrain),type(xtest))
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(xtrain)

xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
word_index = tokenizer.word_index

#print('text example:', xtrain[0])
#print('sequence of indices(before padding):', xtrain_seq[0])
#print('sequence of indices(after padding):', xtrain_pad[0])

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [15]:
emb_dim=300
vocab_len=3543


In [16]:
embedding_matrix = np.zeros((vocab_len, emb_dim))
for word, idx in word_index.items():
    if idx < vocab_len:  # Ensure the index is within bounds
        if word in embed_lookup:
            vector = embed_lookup[word]
            embedding_matrix[idx] = vector
    else:
        print(f"Index {idx} for word '{word}' is out of bounds")

In [17]:
xtrain_pad = np.clip(xtrain_pad, 0, vocab_len - 1)
xtest_pad = np.clip(xtest_pad, 0, vocab_len - 1)

In [18]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
maxlen=10000
vocab_size=vocab_len
from tensorflow.keras.layers import Input, Embedding
EMBEDDING_DIM=300
embedding_dim=300
embedding_layer = Embedding(vocab_len, 
                            EMBEDDING_DIM, 
                            weights=[embedding_matrix], 
                            
                            trainable=False)
print('Trainning embedding model.')

# Testing embedding
#print(embedding_matrix[100])
#print(word_index.get('any'))
#print(data_train[100])
#print(data_test[100])


# Attention
'''
If SINGLE_ATTENTION_VECTOR = true, 
the attention vector is shared across the input_dimensions where the attention is applied.
'''
from keras.layers import multiply
MAX_LENGTH=10000
SINGLE_ATTENTION_VECTOR = False
TIME_STEPS = MAX_LENGTH

def __attention3DBlock__(inputs):
    '''
    Input shape = (batch_size, time_steps, input_dim)
    '''
    input_dim = int(inputs.shape[2])
    a = Permute((2,1))(inputs)  # (batch_size, input_dim, time_steps)
    a = Reshape((input_dim, TIME_STEPS))(a) # (batch_size, input_dim, time_steps)
    a = Dense(TIME_STEPS, activation='softmax')(a) # (batch_size, input_dim, time_steps)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a) # (batch_size, input_dim, time_steps)
    a_probs = Permute((2,1), name='attention_vector')(a) # (batch_size, time_steps, input_dim)
    output_attention_mul = multiply([inputs, a_probs], name='attenton_mul') # (batch_size, time_steps, input_dim)

    return output_attention_mul


# Model
from keras import regularizers
from keras.layers import multiply
#import self_attention

# Parameters
'''
n_x: hidden state size of GRU
regularization: L2 normalization
optimization: AdaGrad/Adam
time_steps = MAX_LENGTH = 270`
'''
batch_size = 32
momentum = 0.9
l2_regularization = 0.001
learning_rate = 0.001
n_x = 32   
epochs = 10
time_steps = MAX_LENGTH

# Build model
print ("Build model...")
sequence_input = Input(shape=(time_steps,), dtype='float32')
print('Sequence input is:', sequence_input) 
embedded_sequences = embedding_layer(sequence_input) 
print('Embedding layer is:', embedded_sequences) 

param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 4, 5],
                  vocab_size=[vocab_size],
                  embedding_dim=[embedding_dim],
                  maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size=10,
                        verbose=False)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv=4, verbose=1, n_iter=5)

L = (GRU(n_x,
            activation='tanh', 
            dropout=0.2, 
            recurrent_dropout=0.2, 
            return_sequences=True,
            kernel_initializer='he_uniform',
            name='Pre-GRU'))(embedded_sequences)
print('GRU is:', L) # (batch_size, time_steps, units=32*2)

L = __attention3DBlock__(L) 
print ('Attention layer is:', L)

L = GRU(n_x, 
        activation='tanh', 
        kernel_regularizer=regularizers.l2(0.001))(L)
print ('Post GRU is:', L)
L = Dense(2, 
          activation='softmax', 
          kernel_regularizer=regularizers.l2(0.001))(L)
print('Dense layer is:', L)

model = Model(inputs=sequence_input, outputs=L)

# Optimization and compile
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
print('Begin compiling...')
model.compile(loss='categorical_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy'])
model.summary()

# Begin training
model.fit(xtrain_pad, 
          np.asarray(ytrain), 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=2,
          validation_data=(xtest_pad, np.asarray(ytest)))
score = model.evaluate(xtest_pad, np.asarray(ytest), batch_size=batch_size)
print ('The evaluation is: ', score)

# Evaluate testing set
#test_accuracy = grid.score(X_test, y_test)


# Save model
print ('Saving model...')
model.save('CNN-GRU-Turkish corpus-200d')

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [26]:
%pip install self_attention

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement self_attention (from versions: none)
ERROR: No matching distribution found for self_attention


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Permute, Reshape, Lambda, RepeatVector, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np

# Hyperparameters and configurations
maxlen = 500
vocab_size = vocab_len  # Adjust this to your vocabulary size
embedding_dim = 300
EMBEDDING_DIM = 300
TIME_STEPS = maxlen
SINGLE_ATTENTION_VECTOR = False
batch_size = 16
epochs = 10
n_x = 32

# Example embedding matrix and data
embedding_matrix = np.random.rand(vocab_size, EMBEDDING_DIM)
xtrain_pad = np.random.randint(0, vocab_size, (1000, maxlen))  # Dummy training data
xtest_pad = np.random.randint(0, vocab_size, (200, maxlen))    # Dummy test data
ytrain = np.random.randint(0, 2, (1000, 2))                    # Dummy training labels
ytest = np.random.randint(0, 2, (200, 2))                      # Dummy test labels

# Embedding layer
embedding_layer = Embedding(vocab_size, 
                            EMBEDDING_DIM, 
                            weights=[embedding_matrix], 
                            input_length=maxlen, 
                            trainable=False)

# Attention mechanism
def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: tf.keras.backend.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

# Build model
sequence_input = Input(shape=(TIME_STEPS,), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)

# First GRU layer
L = GRU(n_x, activation='tanh', dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_initializer='he_uniform', name='Pre-GRU')(embedded_sequences)

# Attention layer
L = attention_3d_block(L)

# Second GRU layer
L = GRU(n_x, activation='tanh', kernel_regularizer=regularizers.l2(0.001))(L)

# Output layer
L = Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(0.001))(L)

# Define and compile model
model = Model(inputs=sequence_input, outputs=L)
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
model.fit(xtrain_pad, ytrain, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(xtest_pad, ytest))

# Evaluate the model
score = model.evaluate(xtest_pad, ytest, batch_size=batch_size)
print('Test evaluation score:', score)

# Save the model
model.save('CNN-GRU-Turkish-corpus-200d.h5')


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 500)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, 500, 300)          │       1,062,900 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Pre-GRU (GRU)                 │ (None, 500, 32)           │          32,064 │ embedding_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_3 (Permute)           │ (None, 32, 500)           │               0 │ Pre-GRU[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_3 (Reshape)           │ (None, 32, 500)           │               0 │ permute_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 32, 500)           │         250,500 │ reshape_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_vec (Permute)       │ (None, 500, 32)           │               0 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention_mul (Multiply)      │ (None, 500, 32)           │               0 │ Pre-GRU[0][0],             │
│                               │                           │                 │ attention_vec[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gru_2 (GRU)                   │ (None, 32)                │           6,336 │ attention_mul[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 2)                 │              66 │ gru_2[0][0]                │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,351,866 (5.16 MB)

 Trainable params: 288,966 (1.10 MB)

 Non-trainable params: 1,062,900 (4.05 MB)

Epoch 1/10
63/63 - 55s - 865ms/step - accuracy: 0.7590 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 2/10
63/63 - 35s - 549ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 3/10
63/63 - 34s - 545ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 4/10
63/63 - 35s - 553ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 5/10
63/63 - 35s - 549ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 6/10
63/63 - 35s - 548ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 7/10
63/63 - 36s - 571ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 8/10
63/63 - 34s - 547ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 9/10
63/63 - 36s - 566ms/step - accuracy: 0.7600 - loss: nan - val_accuracy: 0.7000 - val_loss: nan
Epoch 10/10
63/63 - 34s - 545ms/step - accurac

Test evaluation score: [nan, 0.699999988079071]


In [55]:



    




from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example embedding matrix and word index
# Assuming `embedding_matrix` and `word_index` are defined as in your training script
embedding_matrix = np.random.rand(vocab_size, embedding_dim)
word_index = {w: i for i, w in enumerate(['<PAD>', 'do', 'you', 'believe', 'in', 'god'], 1)}  # Example word index

# Function to convert text to sequence using the word index
def text_to_sequence(text, word_index):
    words = text_to_word_sequence(text)
    sequence = [word_index.get(word, 0) for word in words]  # 0 for unknown words
    return sequence

# Example question
question = "do you believe in god"

# Convert the question to a sequence
q_seq = text_to_sequence(question, word_index)
print(q_seq)

# Pad the sequence
q_pad = pad_sequences([q_seq], maxlen=maxlen, padding='post')

print("Tokenized and Padded Sequence:", q_pad)

# Load the model (if not already loaded)
#model = load_model('CNN-GRU-Turkish-corpus-200d.h5')

# Make predictions
predictions = model.predict(q_pad)

# Print predictions
print("Predictions:", predictions)


[2, 3, 4, 5, 6]
Tokenized and Padded Sequence: [[2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Example question
question = "do you believe in god"

q=[]
for i in question.split():
    q.append(embed_lookup(i))



Tokenized and Padded Sequence: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0